In [5]:
class Node():
    '''
    This class is to be used by DAG, it represetns a single node in a graph
    '''
    def __init__(self,inEdges,outEdges, name,score):
        '''
        :param name
        :param inEdges: list of input edges to a node with thier wieghts. Each edge represented as a list with two elements [prevNode,wieght]
        :param outEdges: same as inEdges but for the output edges. Each element is [nextNode, wieght]
        :param name: int that names the node
        :param score: int that holds the maximum score of the node, calculated by the weights of the largest path to this node
        '''
        self.inEdges = inEdges
        self.outEdges = outEdges
        # bool array that keeps track of if the edge has been traversed.
        # Index of 1 or 0 in inEdgesReady corresponds to the edge in inEdges at the index.
        # 1 means the edge hasn't been traversed, 0 means it has

        self.name = name
        self.score = score

class HMM():

    '''
    class that represents a Hidden Markov Model (HMM)
    contains methods that analyse apsects of the HMM
    '''
    def __init__(self,transMatrix,emitMatrix,emissions):
        '''

        :param transMatrix: Nested dictionary representing a transition matrix
        key:value --> state: RowDictionary
        Where RowDictionary is a dictionary that represents a single row in the matrix.
        key:value --> state: transition pr

        :param emitMatrix: Nested dictionary representing an emission matrix
        key:value --> state: RowDictionary
        Where RowDictionary is a dictionary that represents a single row in the matrix.
        key:value --> state: emission pr

        :param emissions: String representing the emissions from the HMM
        '''

        self.transMatrix = transMatrix
        self.emitMatrix = emitMatrix
        self.emissions = emissions
        self.nodes = self.genNodes()

    def genNodes(self):
        '''
        Generates a dictionary of nodes that represent the HMM
        with key, value pairs name: Node()
        where name is a string that represents the name of the node and Node() is the corresponding node object

        Initializes source node with a score of 1 and all other with score of 0 for probability calculations
        :return:
        '''
        states = self.transMatrix.keys()
        initPr = 1/len(states)
        nodes = {'source':Node([],[[state+str(0),initPr] for state in states],'source',1)}
        for state in states:
            nodes[state+str(0)] = Node([['source',initPr]],[[x+str(1),self.transMatrix[state][x]] for x in states],state+str(0),0)

        for i in range(1,len(self.emissions)-1):
            for state in states:
                nodes[state+str(i)] = Node([[x+str(i-1),self.transMatrix[x][state]] for x in states],[[x+str(i+1),self.transMatrix[state][x]] for x in states],state+str(i),0)

        for state in states:
            nodes[state+str(len(self.emissions)-1)] = Node([[x+str(len(self.emissions)-2),self.transMatrix[x][state]] for x in states],[['sink',0]],state+str(len(self.emissions)-1),0)
        nodes['sink'] = Node([[x+str(len(self.emissions)-1),0] for x in states],[],'sink',0)

        return nodes

    def scoreGraph(self):
        '''
        populates each node in self.nodes with its score.
        score = sum(prevScore * edgewieght * emissionPr) across all nodes in the layer before the current node

        return: score of the sink node
        '''

        states = self.transMatrix.keys()
        for i in range(len(self.emissions)):
            for state in states:
                currentNode = self.nodes[state+str(i)]
                for edge in currentNode.inEdges:
                    prevScore = self.nodes[edge[0]].score
                    newScore = prevScore * edge[1] * self.emitMatrix[currentNode.name[0]][self.emissions[i]]

                    currentNode.score += newScore

        finalScores = [self.nodes[x+str(len(self.emissions)-1)].score for x in states]

        self.nodes['sink'].score = sum(finalScores)
        return self.nodes['sink'].score
import math
def main(fName=''):
    '''
    Handles input/output. Generates transition matrix and path from input data and
    runs prPath to find the probability of the given path
    '''

    with open(fName) as inFile:
        lines = inFile.readlines()
        emissions = lines[0].strip()
        alphabet = lines[2].strip().split()
        states = lines[4].strip().split()

        transMatrix = {}

        # Generate the transition matrix from the given input
        for i in range(len(states)):
            row = lines[7+i].strip().split()

            rowDictionary = {}
            for j in range(len(states)):
                rowDictionary[states[j]] = float(row[j+1])
            transMatrix[row[0]] = rowDictionary
        emitMatrix = {}

        # Generate the emission matrix from the given input
        for i in range(len(states)):
            row = lines[9+len(states)+i].strip().split()
            rowDictionary = {}
            for j in range(len(alphabet)):
                rowDictionary[alphabet[j]] = float(row[j+1])

            emitMatrix[row[0]] = rowDictionary

    hmm = HMM(transMatrix,emitMatrix,emissions)
    print(hmm.scoreGraph())

if __name__ == '__main__':
    main('problem22in.txt')



9.275213483859169e-51


Outcome Likelihood Problem

Given: A string x, followed by the alphabet Σ from which x was constructed, followed by the states States, transition matrix Transition, and emission matrix Emission of an HMM (Σ, States, Transition, Emission).

Return: The probability Pr(x) that the HMM emits x.
